In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import os
import pandas as pd
import numpy as np
from tqdm import trange, tqdm

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

from pandas import read_csv
from scipy import stats

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from tqdm import trange, tqdm


In [2]:
window_size = 192
stride_size = 24
target_window_size = 24
history_size = 150

In [3]:
train_start = '2011-01-01 00:00:00'
train_end = '2014-08-31 23:00:00'
test_start = '2014-08-25 00:00:00' #need additional 7 days as given info
test_end = '2014-09-07 23:00:00'

name = 'LD2011_2014.txt'
save_name = 'elect'
save_path = os.path.join('data', save_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)
csv_path = os.path.join(save_path, name)
if not os.path.exists(csv_path):
    zipurl = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip'
    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(save_path)

data_frame = pd.read_csv(csv_path, sep=";", index_col=0, parse_dates=True, decimal=',')
data_frame = data_frame.resample('1H',label = 'left',closed = 'right').sum()[train_start:test_end]
data_frame.fillna(0, inplace=True) # (32304, 370)

In [16]:
# Select the training data from the DataFrame within the specified date range.
train_data = data_frame[train_start:train_end]

# Select the testing data from the DataFrame within the specified date range.
test_data = data_frame[test_start:test_end]

# Import the MinMaxScaler from sklearn for data normalization.
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler.
scaler = MinMaxScaler()

# Fit the scaler on the training data to learn the scaling parameters.
scaler.fit(train_data)

# Apply the scaler to the training data and create a DataFrame with the same indices and columns.
train_target_df = pd.DataFrame(scaler.transform(train_data), index=train_data.index, columns=train_data.columns)

# Apply the scaler to the testing data and create a DataFrame with the same indices and columns.
test_target_df = pd.DataFrame(scaler.transform(test_data), index=test_data.index, columns=test_data.columns)

# Convert the scaled training DataFrame to a NumPy array for further processing.
train_data = train_target_df.values

# Convert the scaled testing DataFrame to a NumPy array for further processing.
test_data = test_target_df.values


In [5]:
# Define a class for sampling time series data.
class TimeseriesSampler:
    # Initialize the sampler with parameters like size of input and output samples, limit for window sampling, and batch size.
    def __init__(self,
                 timeseries: np.ndarray,
                 insample_size: int=window_size,
                 outsample_size: int=target_window_size,
                 window_sampling_limit: int=history_size * target_window_size,
                 batch_size: int = 8):
        # Store the input time series and other parameters.
        self.timeseries = [ts for ts in timeseries]
        self.window_sampling_limit = window_sampling_limit
        self.batch_size = batch_size
        self.insample_size = insample_size
        self.outsample_size = outsample_size

    # Define an iterator for generating batches of samples.
    def __iter__(self):
        # Continuously generate samples.
        while True:
            # Initialize arrays for input samples and their masks.
            insample = np.zeros((self.batch_size, self.insample_size))
            insample_mask = np.zeros((self.batch_size, self.insample_size))
            # Initialize arrays for output samples and their masks.
            outsample = np.zeros((self.batch_size, self.outsample_size))
            outsample_mask = np.zeros((self.batch_size, self.outsample_size))
            # Randomly select indices of time series to sample from.
            sampled_ts_indices = np.random.randint(len(self.timeseries), size=self.batch_size)
            for i, sampled_index in enumerate(sampled_ts_indices):
                # Select the sampled time series.
                sampled_timeseries = self.timeseries[sampled_index]
                # Randomly choose a point to cut the time series for sampling.
                cut_point = np.random.randint(low=max(1, len(sampled_timeseries) - self.window_sampling_limit),
                                              high=len(sampled_timeseries),
                                              size=1)[0]

                # Extract the input sample from the time series.
                insample_window = sampled_timeseries[max(0, cut_point - self.insample_size):cut_point]
                insample[i, -len(insample_window):] = insample_window
                insample_mask[i, -len(insample_window):] = 1.0
                # Extract the output sample from the time series.
                outsample_window = sampled_timeseries[cut_point:min(len(sampled_timeseries), cut_point + self.outsample_size)]
                outsample[i, :len(outsample_window)] = outsample_window
                outsample_mask[i, :len(outsample_window)] = 1.0
            # Yield the generated input and output samples along with their masks.
            yield insample, insample_mask, outsample, outsample_mask

    # Method to get the last input sample window from each time series.
    def last_insample_window(self):
        # Initialize arrays for the last input sample and its mask.
        insample = np.zeros((len(self.timeseries), self.insample_size))
        insample_mask = np.zeros((len(self.timeseries), self.insample_size))
        for i, ts in enumerate(self.timeseries):
            # Extract the last input sample window from the time series.
            ts_last_window = ts[-self.insample_size:]
            insample[i, -len(ts):] = ts_last_window
            insample_mask[i, -len(ts):] = 1.0
        # Return the last input sample and its mask.
        return insample, insample_mask


In [8]:
# Create a time series data loader for training.
train_loader = TimeseriesSampler(timeseries=train_data.T)

# Define a generic basis function module for the N-Beats model.
class GenericBasis(nn.Module):
    # Initialize the module with backcast and forecast sizes.
    def __init__(self, backcast_size, forecast_size):
        super().__init__()
        self.backcast_size, self.forecast_size = backcast_size, forecast_size

    # Forward pass splits the theta vector into backcast and forecast components.
    def forward(self, theta):
        return theta[:, :self.backcast_size], theta[:, -self.forecast_size:]

# Define a single N-Beats block module.
class NBeatsBlock(nn.Module):
    # Initialize the block with specified sizes for layers and theta vector, and a basis function.
    def __init__(self,
                 input_size,
                 theta_size: int,
                 basis_function: nn.Module,
                 layers: int,
                 layer_size: int):
        super().__init__()
        # Create a sequence of linear layers.
        self.layers = nn.ModuleList([nn.Linear(in_features=input_size, out_features=layer_size)] +
                                      [nn.Linear(in_features=layer_size, out_features=layer_size)
                                       for _ in range(layers - 1)])
        # Linear layer for generating basis parameters.
        self.basis_parameters = nn.Linear(in_features=layer_size, out_features=theta_size)
        # Assign the provided basis function.
        self.basis_function = basis_function

    # Define the forward pass for the N-Beats block.
    def forward(self, x: torch.Tensor):
        # Input to the first layer.
        block_input = x
        # Pass input through each layer, applying ReLU activation function.
        for layer in self.layers:
            block_input = torch.relu(layer(block_input))
        # Compute basis parameters.
        basis_parameters = self.basis_parameters(block_input)
        # Compute backcast and forecast using the basis function.
        return self.basis_function(basis_parameters)

# Define the overall N-Beats model.
class NBeats(nn.Module):
    # Initialize with a list of N-Beats blocks.
    def __init__(self, blocks: nn.ModuleList):
        super().__init__()
        self.blocks = blocks

    # Define the forward pass for the N-Beats model.
    def forward(self, x: torch.Tensor, input_mask: torch.Tensor) -> torch.Tensor:
        # Initialize residuals and flip them for processing.
        residuals = x.flip(dims=(1,))
        # Flip the input mask for alignment with residuals.
        input_mask = input_mask.flip(dims=(1,))
        # Initialize forecast with the last value in the input.
        forecast = x[:, -1:]
        # Iterate through each block, updating residuals and forecast.
        for i, block in enumerate(self.blocks):
            backcast, block_forecast = block(residuals)
            # Update residuals by subtracting backcast, applying mask.
            residuals = (residuals - backcast) * input_mask
            # Update the forecast by adding block forecast.
            forecast = forecast + block_forecast
        return forecast

# Define a function to convert numpy arrays to PyTorch tensors.
def to_tensor(array: np.ndarray):
    return torch.tensor(array, dtype=torch.float32)


In [9]:
# Define the training function for the model.
def train(model, device=torch.device('cuda'), iterations=1000, num_epochs = 1, learning_rate = 1e-3):
    # Initialize the optimizer with Adam algorithm and learning rate.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # List to store loss values after each epoch.
    loss_summary = []
    # Define the loss function as mean squared error.
    loss_fn = F.mse_loss
    # Create an iterator for the training data loader.
    training_set = iter(train_loader)

    # Loop over each epoch.
    for epoch in range(num_epochs):
        # Set the model to training mode.
        model.train()

        # Initialize a progress bar for the number of iterations.
        pbar = trange(iterations)
        # Iterate over each batch in the training data.
        for iteration in pbar:
            # Extract and convert the input and target data to tensors.
            x, x_mask, y, y_mask = map(to_tensor, next(training_set))
            # Reset gradients to zero before starting backpropagation.
            optimizer.zero_grad()

            # Initialize loss to zero.
            loss = torch.zeros(1, device=device, dtype=torch.float32)
            # Perform a forward pass of the model and compute output.
            out = model(x.to(device), x_mask.to(device))
            # Compute the loss between output and actual target.
            loss = loss_fn(out.float(), y.squeeze().to(device).float())

            # Update the progress bar with current loss.
            pbar.set_description(f"Loss:{loss.item()}")
            # Perform backpropagation to compute gradients.
            loss.backward()
            # Update model parameters.
            optimizer.step()

        # Append the last loss value to the summary list.
        loss_summary.append(loss.cpu().detach())

    # Return the loss summary and the optimizer.
    return loss_summary, optimizer

# Define the evaluation function for the model.
def evaluate(model, optimizer, device=torch.device('cuda')):
    # List to store forecasted values.
    forecasts = []
    # Calculate the number of test windows.
    test_windows = test_data.T.shape[1] // target_window_size

    # Disable gradient calculations for evaluation.
    with torch.no_grad():
        # Set the model to evaluation mode.
        model.eval()
        # Iterate over each test window.
        for i in trange(test_windows):
            # Combine training and test data up to the current window.
            window_input_set = np.concatenate([train_data.T, test_data.T[:, :i * target_window_size]], axis=1)
            # Create a time series sampler for the combined data.
            input_set = TimeseriesSampler(timeseries=window_input_set)
            # Extract the last in-sample window.
            x, x_mask = map(to_tensor, input_set.last_insample_window())
            # Get model predictions for the current window.
            window_forecast = model(x.to(device), x_mask.to(device)).cpu().detach().numpy()
            # Append the forecast to the forecasts list.
            forecasts = window_forecast if len(forecasts) == 0 else np.concatenate([forecasts, window_forecast], axis=1)

    # Return the root mean square error between forecasts and actual test data.
    return np.sqrt(np.mean((forecasts - test_data.T) ** 2))


In [17]:
model = NBeats(nn.ModuleList([NBeatsBlock(input_size=window_size,
                                           theta_size=window_size + target_window_size,
                                           basis_function=GenericBasis(backcast_size=window_size,
                                                                       forecast_size=target_window_size),
                                           layers=4,
                                           layer_size=512)
                                   for _ in range(30)])).cuda()

loss, optimizer = train(model, num_epochs=3)
model_save_path = '/content/drive/MyDrive/Colab_Notebooks/ads_506/nbeats.pth'
torch.save(model.state_dict(), model_save_path)

Loss:0.004569113254547119: 100%|██████████| 1000/1000 [00:52<00:00, 19.13it/s]


In [19]:
evaluate(model, optimizer)

100%|██████████| 14/14 [00:00<00:00, 20.49it/s]


0.06692914282696105